In [2]:
import pandas as pd
from lexical_diversity import lex_div as ld
import json
import math

In [3]:
df = pd.read_csv('data/tcc_ceds_music.csv', index_col=0, encoding = 'unicode_escape')

cols = [3, 4, 28]
df = df[df.columns[cols]]
df

,genre,lyrics,topic
0,pop,hold time feel break feel untrue convince spea...,sadness
4,pop,believe drop rain fall grow believe darkest ni...,world/life
6,pop,sweetheart send letter goodbye secret feel bet...,music
10,pop,kiss lips want stroll charm mambo chacha merin...,romantic
12,pop,till darling till matter know till dream live ...,romantic
...,...,...,...
82447,hip hop,cause fuck leave scar tick tock clock come kno...,obscene
82448,hip hop,minks things chain ring braclets yap fame come...,obscene
82449,hip hop,get ban get ban stick crack relax plan attack ...,obscene
82450,hip hop,check check yeah yeah hear thing call switch g...,obscene


In [8]:
def process_prompt(prompt):
    if prompt.startswith("Generate music lyrics"):
        res = prompt.split("for genre ", 1)[1].split(", topic")[0]
    elif prompt.startswith("Generate lyrics in genre"):
        res = prompt.split("lyrics in genre ", 1)[1].split(" about")[0]
    elif prompt.startswith("For genre"):
        res = prompt.split("For genre ", 1)[1].split(" generate")[0]
    else:
        res = prompt.split("Generate ", 1)[1].split(" lyrics")[0]
    return res

In [11]:
f = open('generation_output.json')
data = json.load(f)
data.pop(0)
#genres = {'pop': '', 'country': '', 'blues': '', 'jazz': '', 'reggae': '', 'hip hop': ''}
genres = {'pop': '', 'country': '', 'blues': '', 'jazz': '', 'reggae': '', 'hip hop': '', 'rock': ''}
counter = {'pop': 0, 'country': 0, 'blues': 0, 'jazz': 0, 'reggae': 0, 'hip hop': 0, 'rock': 0}
threshold = 100
for i in data:
    prompt = i[0]
    prediction = i[1]
    res = process_prompt(prompt)
    if (counter[res] < threshold):
        genres[res] = genres[res] + prediction[len(prompt):]
        counter[res] = counter[res] + 1


relative_freqs = {}
for genre, lyrics in genres.items():
    flt = ld.flemmatize(lyrics)

    relative_freq_genre = {}
    for word in flt:
        if (word!=''):
            freq_num = lyrics.count(word)/len(lyrics)
            lyrics_in_dataset = ''.join(df.loc[df['genre'] == genre].lyrics)
            freq_denom = lyrics_in_dataset.count(word)/len(lyrics_in_dataset)
            if freq_denom != 0:
                relative_freq_genre[word] = freq_num/freq_denom
    
    sorted_freqs = sorted(relative_freq_genre)
    relative_freqs[genre] = sorted_freqs[:10]

print(relative_freqs)

{'pop': ['about', 'act', 'admit', 'afraid', 'ahead', 'ahh', 'ahhh', 'al', 'alive', 'alright'], 'country': ['about', 'afraid', 'agony', 'ahead', 'alive', 'alright', 'angel', 'anybody', 'anymore', 'apart'], 'blues': ['aaaah', 'about', 'ache', 'advice', 'afraid', 'age', 'aha', 'alive', 'alright', 'anybody'], 'jazz': ['about', 'adventure', 'afraid', 'ahead', 'alive', 'alright', 'anymore', 'apart', 'arm', 'asleep'], 'reggae': ['afraid', 'ahaaa', 'ahead', 'aight', 'alive', 'alright', 'angel', 'answer', 'anybody', 'anymore'], 'hip hop': ['a', 'accept', 'ahahah', 'ahead', 'aight', 'album', 'alive', 'alright', 'american', 'answer'], 'rock': ['a', 'about', 'advent', 'aflame', 'afraid', 'afternoon', 'age', 'ahhhh', 'alive', 'alright']}
